# k-Nearest Neigbours (kNN)

## kNN Classification

In [1]:
import sklearn.datasets
import sklearn.model_selection
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
iris_data = sklearn.datasets.load_iris(as_frame=True)

In [ ]:
target_num2name = {ix: str(name) for ix, name in enumerate(iris_data['target_names'])}
feature_names = iris_data['feature_names']

In [ ]:
iris_df = iris_data['frame']
iris_df['named_target'] = iris_df['target'].map(target_num2name)
iris_df.head(20)

In [ ]:
train, test = sklearn.model_selection.train_test_split(iris_df)
train.shape, test.shape

In [ ]:
import sklearn.neighbors as sn

Nearest Neighbours -> k

In [ ]:
# n_neighbours is {k}
knn = sn.KNeighborsClassifier(n_neighbors=1)
knn.fit(X = train[feature_names], y=train['named_target'])

In [ ]:
test_pred = knn.predict(test[feature_names])

In [ ]:
import sklearn.metrics as skm

In [ ]:
skm.accuracy_score(
    y_true = test['named_target'], 
    y_pred = test_pred
)

In [ ]:
# Create the confusion matrix
cm = skm.confusion_matrix(test['named_target'], test_pred)

# Display it
disp = skm.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_num2name.values())
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
print(skm.classification_report(y_true=test['named_target'], y_pred=test_pred))

#### Aufgabe 

- Evaluieren Sie den `skm.accuracy_score` für das `test` und `trainings`-Set für verschiedene Werte von k $\in [1, 2, 5, 10, 20, 40]$
- Speichern Sie die Ergebnisse in 2 dictionaries `score_train` und `score_test` die ein Mapping von dem Wert `k` zu der `score` erhalten

## kNN - Regression

In [ ]:
train, test = sklearn.model_selection.train_test_split(iris_df, random_state=23)

#### Aufgabe

- Nutzen Sie den `sn.KNeighborsRegressor` umd das Target `petal width` auf Basis des Feature `petal length` vorherzuagen
  - Evaluieren Sie den `skm.mean_absolute_error` für Werte von k $\in [1, 5, 10, 20, 40, 75]$
- Führen Sie auch eine Regression auf das Target `petal width` auf Basis der 3 Feature `petal length`, `sepal width`, `sepal length` durch